In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Citation to the original paper:

1.   Al-Rabeah, M.H., Lakizadeh, A. Prediction of drug-drug interaction events using graph neural networks based feature extraction. Sci Rep 12, 15590 (2022). https://doi.org/10.1038/s41598-022-19999-4

Link to the original paper’s repo: https://github.com/Mohammad-Hussain95/GNN_DDI


Link to Video: https://mediaspace.illinois.edu/media/t/1_gg81lr8i

Link to public repo: https://github.com/genggeng88/dl4h_team61


# Introduction

* Background of the problem
  * Type of problem:
    * Nowadays, many difficult diseases are treated using drug mixes (Polypharmacy), which is a good approach that utilizes the synergistic effects of drug interactions. However, unplanned DDIs could risk a patient's life because they may cause side effects or perhaps dangerous toxicity. The detection of DDIs becomes much more necessary, but diagnosing DDI on a large number of drug pairs, both in vitro and in vivo, is costly and time-consuming.
  * Importance/meaning of solving the problem:
    * Detecting possible DDIs decreases the incidence of unexpected drug interactions and reduces drug production costs. It also can optimize the drug creation process.
  * Difficulty of the problem:
    *  DDI network can provide vital information about drugs interactions. Furthermore, using an attributed heterogeneous DDIs network that presents the drug's interaction types along with the drug features can better demonstrate the intrinsic characteristics of a drug. However, it is challenging to integrate various features effectively because the drug features might be correlated and contain redundant information.
  * State of the art methods and effectiveness:
    * There are four popular approaches in the DDI prediction field: Similarity-based methods, Matrix Factorization-based methods, network analysis-based methods and Deep Learning-based methods. Most of current methods are developed to predict whether drugs interact or not, but not to predict the DDI events. Even though few researches created strong efforts in event prediction but there is a space for advancement.
*  Paper explanation
  * What did the paper propose:
    * The paper proposed a method for predicting DDI and their type (event) based on attributed heterogeneous graph embedding and a deep learning approach.
  * What is the innovations of the method:
    * This paper first built a heterogeneous drug network by integrating the drug properties in each type of DDI. It then made predictions on what kind of interaction is between drugs.
  * How well the proposed method work:
    * Models like DDIMDL, CNN-DDI, DANN-DDI, MDNN, RF, KNN, LR were used for performance comparison with GNN-DDI base on the metrics like ACC, AUPR, AUC, F1 Score, Precision, and Recall. GNN-DDI showed the best performance among different models.
  * What is the contribution to the reasearch regime:
    * This paper proposed a new approach in generating heterogeneous drug network. It also helped make predictions on not only drug interaction, but also drug interaction events which can be beneficial to patients and drug production.


# Scope of Reproducibility:

1. Impact of Embedding Dimension Size:
    - Hypothesis: The size of the embedding dimension affects the model's performance, with a dimension size of 32 leading to the best accuracy. Dimension sizes of 16 and 64 yield slightly inferior performance.
    - Method: Train the model using different embedding dimension sizes (16, 32, 64) and compare their performance metrics.

3. Impact of Different Drug Feature Matrices:
    - Hypothesis: Using different drug feature matrices (similarity matrices) displays varying performance on the proposed model. The combined feature matrix shows the best performance.
    - Method: Test the model's performance using different drug feature matrices and assess the effectiveness of each matrix in predicting DDIs.
4. Comparison with Existing Models:
    - Hypothesis: The proposed method outperforms existing models such as DNN, RF, KNN, and LR in predicting DDIs.
    - Method: Compare the performance of the proposed method with existing models (DNN, RF, KNN, LR) using common performance metrics and assess its superiority.

# Methodology

Environment:
*   Python 3.6.12
*   Tensorflow 2.1
*   keras 2.3
*   numpy 1.19
*   pandas 1.1.3
*   sqlite3 3.8.6
*   tqdm 4.63
*   matplotlib 3.3



In [ ]:
# importing packages
import numpy as np
import sys
import os
import sqlite3
from google.colab import drive
sys.path.append(os.path.abspath('/content/drive/MyDrive'))

##   Process Overview

The proposed model contains two stages:

1. Collect drug information from different sources and integrate them through the formation of attributed heterogeneous networks. Then use a Graph Neural Network to generate embedding vectors based on different drug interaction types and drug attributes.

The following figure provides an overview on the first stage:<br>
<img src="https://drive.google.com/uc?export=view&id=1uda8ODPHVBPSWiFK62fiQc1Qij2XsN-M" width="450" height="300">

2. Aggregate the representation vectors and use a fully connected neural network to predict the DDIs.

The following figure provides an overview on the second stage:<br>
<img src="https://drive.google.com/uc?export=view&id=1BbsygQJHpvRCFetVN67ebZixQTLMJBRd" width="450" height="300">


##  Data
In this project, we obtained the data from a [sqlite3 database](https://github.com/YifanDengWHU/DDIMDL) which was compiled from [DrugBank](https://go.drugbank.com/) 5.1.3 verision. It has 4 tables:
1. <b>drug</b> contains 572 kinds of drugs and their target, enzyme, pathway, and SMILES features.
2. <b>event</b> contains the 37264 DDIs between the 572 kinds of drugs.
3. <b>extraction</b> is the process result of NLPProcess. Each interaction is transformed to a tuple: {mechanism, action, drugA, drugB}
4. <b>event_numer</b> lists the kinds of DDI events and their occurence frequency.



We first import the data from the paper's provided database.

In [ ]:
# Upload the tables from sqlite3 database called "Event.db"

#Connection to the DB
conn = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/event.db')

cursor = conn. cursor()
cursor. execute("SELECT name FROM sqlite_master WHERE type='table';") # [('event_number',), ('event',), ('drug',), ('extraction',)]
print("Tables : ",cursor. fetchall())
cursor. execute("SELECT name FROM PRAGMA_TABLE_INFO('drug');") # [('index',), ('id',), ('target',), ('enzyme',), ('pathway',), ('smile',), ('name',)]
print("drug : ",cursor. fetchall())
cursor. execute("SELECT name FROM PRAGMA_TABLE_INFO('event');") # [('index',), ('id',), ('target',), ('enzyme',), ('pathway',), ('smile',), ('name',)]
print("event : ",cursor. fetchall())
cursor. execute("SELECT COUNT(*) FROM event") # [('index',), ('id',), ('target',), ('enzyme',), ('pathway',), ('smile',), ('name',)]
print("event row COUNT: ",cursor. fetchall())

# close the DB connection
conn.close()

For each drug feature in the data, we generate a similarity matrix to be used as an attribute in our Drug to Drug interaction networks.

In [ ]:
# Generate similarity matrices
import csv
import pandas as pd
from pandas import DataFrame
from sklearn.decomposition import PCA

def save_f(name,mat):
  print(name)
  print(mat.shape)
  df = pd.DataFrame(mat)
  df.to_csv('/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/'+str(name)+'.csv', header=None, index=False)

# construct similarity matrices from adjacency matrices of the properties useing the Jacquard similarity function
def Jaccard(matrix):
  matrix = np.mat(matrix)
  numerator = matrix * matrix.T
  denominator = np.ones(np.shape(matrix)) * matrix.T + matrix * np.ones(np.shape(matrix.T)) - matrix * matrix.T
  return numerator / denominator


# generate the feature vector
def feature_vector(df, feature_name):
  all_feature = []
  drug_list = np.array(df[feature_name]).tolist()
  # Features for each drug, for example, when feature_name is target, drug_list=["P30556|P05412","P28223|P46098|……"]
  for i in drug_list:
    for each_feature in i.split('|'):
      if each_feature not in all_feature:
        all_feature.append(each_feature)  # obtain all the features
  feature_matrix = np.zeros((len(drug_list), len(all_feature)), dtype=float)
  df_feature = DataFrame(feature_matrix, columns=all_feature)  # Consrtuct feature matrices with key of dataframe
  for i in range(len(drug_list)):
    for each_feature in df[feature_name].iloc[i].split('|'):
      df_feature[each_feature].iloc[i] = 1

  sim_matrix = np.asarray(Jaccard(np.array(df_feature)))
  sim_matrix1 = np.array(sim_matrix)
  pca = PCA(n_components=len(sim_matrix1))  # PCA dimension
  pca.fit(sim_matrix)
  sim_matrix = pca.transform(sim_matrix)

  return sim_matrix

conn = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/event.db')
df_drug = pd.read_sql('select * from drug;', conn)

feature_list = ['target', 'enzyme', 'pathway', 'smile']

print(df_drug[feature_list[0]][:2])
print(df_drug[:][:2])

drugs = df_drug[:]
drugs = np.array(np.vstack((df_drug['index'],df_drug['name'])))
drugs = drugs.T

# save the four individual feature matrix
for feature in feature_list:
  mat = feature_vector(df_drug, feature)
  save_f(feature+"_PCA", mat)

conn.close()

Using our event and extraction data, we create the Drug-Drug Interaction network that will be used to create Drug-to-Event-type embedding vectors for each feature.

In [ ]:
# retrieve records from the extraction table in the database
# and construct the DDI Matrix

import time
import matplotlib.pyplot as plt
import random
from tqdm import tqdm
import itertools

conn = sqlite3.connect('/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/event.db')
extraction = pd.read_sql('select * from extraction;', conn)
mechanism = extraction['mechanism']
action = extraction['action']
drugA = extraction['drugA']
drugB = extraction['drugB']
d_label = {}
d_event=[]
for i in range(len(mechanism)):
  d_event.append(mechanism[i]+" "+action[i])
count={}
for i in d_event:
  if i in count:
    count[i]+=1
  else:
    count[i]=1
list1 = sorted(count.items(), key=lambda x: x[1],reverse=True)
for i in range(len(list1)):
  d_label[list1[i][0]]=i

DDI=[]
for i in range(len(d_event)):
  DDI.append(np.hstack((d_label[d_event[i]],drugA[i], drugB[i])))

mat_DDI = np.array(DDI)
key = drugs[:,1]
val = drugs[:,0]
dic = dict(zip(key,val))
postive1 = [dic[item] for item in mat_DDI[:,1]]
postive2 = [dic[item] for item in mat_DDI[:,2]]
full_pos = np.array(np.vstack((mat_DDI[:,0],postive1,postive2))).astype('int32')
full_pos = full_pos.T

df = pd.DataFrame(np.array(full_pos).tolist())
df.to_csv('/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/full_pos2.txt', header=None, index=None, sep=' ')

conn.close()

Here we generate negative drug pairings for use in training the Graph Neural Network.

In [ ]:
# generate negative pairs (non-interacting drug pairs) from a given matrix of positive pairs (interacting drug pairs).

def make_neg_pairs(matrix):
  all_pos = np.array(matrix)
  s1=np.unique(all_pos[:,0])
  s2=np.unique(all_pos[:,1])
  s3=set(s1).union(s2)
  conncted_drug = sorted(s3)
  print("there are ", len(s3), " drugs have connection out of 572")
  ss = [ii for ii in range(572) if not ii in s3]
  print("there are ", len(ss), " drugs without connection out of 572")
  pairs_false = list()
  pairs = list()
  comparing = all_pos
  print("start callcolate combinations ... ")
  for dr1,dr2 in itertools.combinations(conncted_drug,2):
    d1=np.array([dr1,dr2])
    d2=np.array([dr1,dr2])
    if dr1 == dr2: continue
    else: pairs.append((dr1,dr2))
  print("all pairs : ",len(pairs))
  for dr in tqdm(pairs, desc="pairs_false generating : "):
    d1=np.array([dr[0],dr[1]])
    d2=np.array([dr[1],dr[0]])
    if not (dr[0]==dr[1]):
      if not ((d2 == comparing).all(axis=1).any() or (d1 == comparing).all(axis=1).any()):
        pairs_false.append([dr[0],dr[1]])
  base=[]
  base2=[]
  for o in tqdm(pairs_false, "all_neg generating : "):
    if (not any(o[0] in h for h in base)) or (not any(o[1] in h for h in base)):
      base.append(o)
    else:
      base2.append(o)
  if len(base) > len(conncted_drug) :
    print("less base .... !")
  pairs_f1 = np.array(base2)
  np.random.shuffle(pairs_f1)
  all_neg = np.concatenate((base,pairs_f1[:len(all_pos)-len(base)]),axis=0)
  np.random.shuffle(all_neg)
  print("all_neg.shape : ", all_neg.shape, "all_pos.shape : ", all_pos.shape)
  df = pd.DataFrame(np.array(all_neg).tolist())
  df.to_csv('/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/all_neg2.txt', header=None, index=None, sep=' ')
  return all_neg

all_neg = make_neg_pairs(full_pos[:,1:])

Next we prepare train, test, and validation subsets for our GNN model Training.

In [ ]:
# prepare the train and test subsets

for itm in tqdm(range(9)):
  if itm == 0:
    full_pos = np.array(np.array(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/full_pos2.txt", header=None , sep=' ')).tolist())
  elif itm == 1:
    all_neg = np.array(np.array(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/all_neg2.txt", header=None , sep=' ')).tolist())
  elif itm == 2:
    target = np.array(np.array(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/target_PCA.csv", header=None)).tolist())
    enzyme = np.array(np.array(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/enzyme_PCA.csv", header=None)).tolist())
    pathway = np.array(np.array(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/pathway_PCA.csv", header=None)).tolist())
    smile = np.array(np.array(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/smile_PCA.csv", header=None)).tolist())
  elif itm == 3:
    full_pos = np.array(np.vstack((full_pos[:,0],full_pos[:,1],full_pos[:,2],[1]*len(full_pos)))).astype('int32').T
    all_cat_pos = []
    for i in range(65):
      all_cat_pos.append(([np.array(item).tolist() for item in full_pos if item[0]==i]))
  elif itm == 4:
    l_l = len(all_cat_pos[0])
    f_l = 0
    all_cat_neg = []
    for i in range(65):
      all_cat_neg.append(np.vstack(([i]* len(all_cat_pos[i]),all_neg[f_l:l_l,0].tolist(),all_neg[f_l:l_l,1].tolist(),[0]* len(all_cat_pos[i]))).T)
      f_l = l_l
      if i<64:
        l_l += len(all_cat_pos[i+1])
  elif itm == 5:
    train_cat_pos = []
    train_cat_neg = []
    valid_cat_pos = []
    valid_cat_neg = []
    test_cat_pos = []
    test_cat_neg = []
    for i in range(65):
      train_cat_pos.append((all_cat_pos[i][:(len(all_cat_pos[i])*65//100)]))
      train_cat_neg.append((all_cat_neg[i][:(len(all_cat_neg[i])*65//100)]))
      valid_cat_pos.append((all_cat_pos[i][(len(all_cat_pos[i])*65//100):(len(all_cat_pos[i])*80//100)]))
      valid_cat_neg.append((all_cat_neg[i][(len(all_cat_neg[i])*65//100):(len(all_cat_neg[i])*80//100)]))
      test_cat_pos.append((all_cat_pos[i][(len(all_cat_pos[i])*80//100):]))
      test_cat_neg.append((all_cat_neg[i][(len(all_cat_neg[i])*80//100):]))
    train_cat_pos1 = np.array([ii for i in train_cat_pos for ii in i ])
    train_cat_neg1 = np.array([ii for i in train_cat_neg for ii in i ])
    valid_cat_pos1 = np.array([ii for i in valid_cat_pos for ii in i ])
    valid_cat_neg1 = np.array([ii for i in valid_cat_neg for ii in i ])
    test_cat_pos1 = np.array([ii for i in test_cat_pos for ii in i ])
    test_cat_neg1 = np.array([ii for i in test_cat_neg for ii in i ])
  elif itm == 6:
    # train_cat = np.array(np.vstack((train_cat_pos1,train_cat_neg1)))
    valid_cat = np.array(np.vstack((valid_cat_pos1,valid_cat_neg1)))
    test_cat = np.array(np.vstack((test_cat_pos1,test_cat_neg1)))
    # train_final = np.array(train_cat[:,:3])
    train_final = np.array(train_cat_pos1[:,:3])
  elif itm == 7:
    m1 = np.array(target).astype(np.float64)
    m2 = np.array(enzyme).astype(np.float64)
    m3 = np.array(pathway).astype(np.float64)
    m4 = np.array(smile).astype(np.float64)
    f_all_m1 = np.array(np.column_stack((drugs[:,0],m1)))
    f_all_m2 = np.array(np.column_stack((drugs[:,0],m2)))
    f_all_m3 = np.array(np.column_stack((drugs[:,0],m3)))
    f_all_m4 = np.array(np.column_stack((drugs[:,0],m4)))
    print(len(f_all_m1[:]),len(f_all_m1[0]))
  else:
    print("\n################# DDI copmleted ##################")
    print("################# featuers copmleted ##################")

print(test_cat_pos1.shape,valid_cat_pos1.shape,train_cat_pos1.shape,train_cat_pos1[0],valid_cat_pos1[0])
tr = []
print(" event >> valid : test >> whate events in valid : whate events in test ")
for i in range(572):
  s1 = len(train_cat_pos1[np.where(train_cat_pos1[:,1:]==i)])
  s2 = len(valid_cat_pos1[np.where(valid_cat_pos1[:,1:]==i)])
  s3 = len(test_cat_pos1[np.where(test_cat_pos1[:,1:]==i)])
  if s1 == 0:
    vid = valid_cat_pos1[np.where(valid_cat_pos1[:,1:]==i)[0],:].tolist()
    tst = test_cat_pos1[np.where(test_cat_pos1[:,1:]==i)[0],:].tolist()
    print(i," >>   ",s2," : ",s3 ," >> "
    ,np.unique(valid_cat_pos1[np.where(valid_cat_pos1[:,1:]==i)[0],0])
    ," : ",np.unique(test_cat_pos1[np.where(test_cat_pos1[:,1:]==i)[0],0])
    ," >>   ",vid," : ",tst )
    if not np.isnan(vid).all():
      for item in vid:
        tr.append(item)
    if not np.isnan(tst).all():
      for item in tst:
        tr.append(item)

print(" missing sample in train : " , tr)

print(" event >> test : valid")
for i in range(572):
  s4 = len(test_cat_neg1[np.where(test_cat_neg1[:,1:]==i)])
  s5 = len(valid_cat_neg1[np.where(valid_cat_neg1[:,1:]==i)])
  if s4 == 0 or s5 == 0:
    print(i," >>   ",s4," : ",s5 )

s1, s2, s3, s4, s5 ,s_all ,l_all ,persnt ,sal= [], [], [], [], [], [], [], [], 0
events = np.unique(full_pos[:,0])
for i in events:
  if i < 6:
    pp = (len(full_pos[np.where(full_pos[:,0]==i)])/len(full_pos))*100
    persnt.append(round(pp,1))
    l_all.append(""+str(round(pp,1))+"%")
    s_all.append(len(full_pos[np.where(full_pos[:,0]==i)]))
  else:
    sal += len(full_pos[np.where(full_pos[:,0]==i)])
  s1.append(len(train_cat_pos1[np.where(train_cat_pos1[:,0]==i)]))
  s2.append(len(valid_cat_pos1[np.where(valid_cat_pos1[:,0]==i)]))
  s3.append(len(test_cat_pos1[np.where(test_cat_pos1[:,0]==i)]))
  s4.append(len(test_cat_neg1[np.where(test_cat_neg1[:,0]==i)]))
  s5.append(len(valid_cat_neg1[np.where(valid_cat_neg1[:,0]==i)]))
  # print(i," >> ",s1," : ",s2 ," : ",s3 ," : ",s4 ," : ",s5 )
s_all.append(sal)
persnt.append(round(sal/len(full_pos)*100,1))
l_all.append(""+str(persnt[-1])+"%")

Here we can visualiize the number of samples we have for each interaction/event type.

In [ ]:
# Statistics of datasets
plt.title("number of samples in the events")
plt.plot(events, s1, label='data train_pos + ')
plt.plot(events, s2, label='data valid_pos + ')
plt.plot(events, s3, label='data test_pos + ')
plt.plot(events, s4, label='data test_neg - ')
plt.plot(events, s5, label='data valid_neg - ')
plt.xlabel('event')
plt.ylabel('number of samples')
plt.legend()
plt.show()

In [ ]:
# print('\n',tr,'\n',vid,'\n',tst)
print(train_final.shape)
tr1 = np.array(tr)

# plt.title("number of samples in the events")
# plt.bar(l_all, s_all)
print(s_all," | sum : ",sum(s_all)," all : ",len(full_pos),"\n",persnt," | ",sum(persnt))

ddd = np.zeros((7)).tolist()
ddd[0] = 0.1
myexplode = ddd

labels = ["event "+str(i+1)+" : "+str(j) for i,j in enumerate(l_all)]
labels[-1] = "events 7-65 : "+str(l_all[-1])
# title = plt.title("number of samples in the events")
# title.set_ha("center")
plt.gca().axis("equal")
pie = plt.pie(persnt, labels = l_all, explode = myexplode, startangle=90)
plt.legend(pie[0],labels, bbox_to_anchor=(0.83,0.5), loc="center", fontsize=10, bbox_transform=plt.gcf().transFigure)
plt.subplots_adjust(left=0.0, bottom=0.1, right=0.6)
# image_format = 'svg' # e.g .png, .svg, etc.
# image_name = 'event_pie.svg'
# plt.savefig(image_name, format=image_format, dpi=1200)

print(tr1.shape, tr1)
train_final1 = np.concatenate((train_final,tr1[:,:3]))
print(train_final1.shape,train_final1[-1])
train_final = train_final1

In [ ]:
# Save all the proprecessed data into .txt files for next step using
df = pd.DataFrame(np.array(train_final))
df.to_csv('/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/train.txt', header=None, index=None, sep=' ')
df = pd.DataFrame(np.array(valid_cat))
df.to_csv('/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/valid.txt', header=None, index=None, sep=' ')
df = pd.DataFrame(np.array(test_cat))
df.to_csv('/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/test.txt', header=None, index=None, sep=' ')

def write_f(a_f,path_f):
  print(a_f.shape)
  a,b = a_f.shape
  b = b-1
  with open(path_f, "w") as txt_file:
    csv.writer(txt_file, delimiter=' ').writerow([a,b])
    csv.writer(txt_file, delimiter=' ').writerows(a_f)

print(len(f_all_m1[:]),len(f_all_m1[0]), f_all_m1.shape)
f1 = write_f(f_all_m1,'/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/featuers_m1.txt')
f2 = write_f(f_all_m2,'/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/featuers_m2.txt')
f3 = write_f(f_all_m3,'/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/featuers_m3.txt')
f4 = write_f(f_all_m4,'/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/featuers_m4.txt')

With our feature networks and training splits created, we are ready to create and train our GNN.

## Model

In order to make predictions on DDI type, we need to first use a Graph Neural Network to generate a drug embedding vector based on different drug interaction types and drug attributes, and then aggregate the representation vectors. The drug embedding vectors and the aggregation are implemented as follows:

### Graph Neural Network

In [ ]:
import math
import os
import sys
import time
from pandas import DataFrame
import pandas as pd
import csv
import numpy as np
import tensorflow as tf
from numpy import random
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

This function generates batches of data from pairs and their corresponding neighbors, ensuring a specified batch_size. It iterates through the data, constructing batches of x, y, t, and neigh arrays, which are then yielded for use in model training or evaluation.

In [ ]:
def get_batches(pairs, neighbors, batch_size):
    n_batches = (len(pairs) + (batch_size - 1)) // batch_size

    for idx in range(n_batches):
        x, y, t, neigh = [], [], [], []
        for i in range(batch_size):
            index = idx * batch_size + i
            if index >= len(pairs):
                break
            x.append(pairs[index][0])
            y.append(pairs[index][1])
            t.append(pairs[index][2])
            neigh.append(neighbors[pairs[index][0]])
        yield (np.array(x).astype(np.int32), np.array(y).reshape(-1, 1).astype(np.int32), np.array(t).astype(np.int32), np.array(neigh).astype(np.int32))

The train_model function trains a Graph Neural Network (GNN) model using TensorFlow. It constructs the computational graph, and optimizes the model using Adam optimizer. During training, it creates node sequences using Random Walk and learns embeddings through Skip-Gram. Here we iterate through epochs, evaluating and updating the model's performance. Finally, it stores the trained model embeddings and returns evaluation metrics.

In [ ]:
#GNN training function
def train_model(network_data, feature_dic, log_name, f_num, file_name):
    args=parse_args()
    vocab, index2word, train_pairs = generate(network_data, args.num_walks, args.walk_length, args.schema, file_name, args.window_size, args.num_workers, args.walk_file)

    edge_types = list(network_data.keys())

    num_nodes = len(index2word)
    edge_type_count = len(edge_types)
    epochs = args.epoch
    batch_size = args.batch_size
    embedding_size = args.dimensions # Dimension of the embedding vector.
    embedding_u_size = args.edge_dim
    u_num = edge_type_count
    num_sampled = args.negative_samples # Number of negative examples to sample.
    dim_a = args.att_dim
    att_head = 1
    neighbor_samples = args.neighbor_samples

    neighbors = generate_neighbors(network_data, vocab, num_nodes, edge_types, neighbor_samples)

    graph = tf.Graph()

    if feature_dic is not None:
        feature_dim = len(list(feature_dic.values())[0])
        print('feature dimension: ' + str(feature_dim))
        features = np.zeros((num_nodes, feature_dim), dtype=np.float32)
        for key, value in feature_dic.items():
            if key in vocab:
                features[vocab[key].index, :] = np.array(value)

    with graph.as_default():
        global_step = tf.Variable(0, name='global_step', trainable=False)

        if feature_dic is not None:
            node_features = tf.Variable(features, name='node_features', trainable=False)
            feature_weights = tf.Variable(tf.truncated_normal([feature_dim, embedding_size], stddev=1.0))

            embed_trans = tf.Variable(tf.truncated_normal([feature_dim, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))
            u_embed_trans = tf.Variable(tf.truncated_normal([edge_type_count, feature_dim, embedding_u_size], stddev=1.0 / math.sqrt(embedding_size)))
        else:
            node_embeddings = tf.Variable(tf.random_uniform([num_nodes, embedding_size], -1.0, 1.0))
            node_type_embeddings = tf.Variable(tf.random_uniform([num_nodes, u_num, embedding_u_size], -1.0, 1.0))

        trans_weights = tf.Variable(tf.truncated_normal([edge_type_count, embedding_u_size, embedding_size // att_head], stddev=1.0 / math.sqrt(embedding_size)))
        trans_weights_s1 = tf.Variable(tf.truncated_normal([edge_type_count, embedding_u_size, dim_a], stddev=1.0 / math.sqrt(embedding_size)))
        trans_weights_s2 = tf.Variable(tf.truncated_normal([edge_type_count, dim_a, att_head], stddev=1.0 / math.sqrt(embedding_size)))
        nce_weights = tf.Variable(tf.truncated_normal([num_nodes, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([num_nodes]))

        # Input data
        train_inputs = tf.placeholder(tf.int32, shape=[None])
        train_labels = tf.placeholder(tf.int32, shape=[None, 1])
        train_types = tf.placeholder(tf.int32, shape=[None])
        node_neigh = tf.placeholder(tf.int32, shape=[None, edge_type_count, neighbor_samples])

        # Look up embeddings for nodes
        if feature_dic is not None:
            node_embed = tf.nn.embedding_lookup(node_features, train_inputs)
            node_embed = tf.matmul(node_embed, embed_trans)
        else:
            node_embed = tf.nn.embedding_lookup(node_embeddings, train_inputs)

        if feature_dic is not None:
            node_embed_neighbors = tf.nn.embedding_lookup(node_features, node_neigh)
            node_embed_tmp = tf.concat([tf.matmul(tf.reshape(tf.slice(node_embed_neighbors, [0, i, 0, 0], [-1, 1, -1, -1]), [-1, feature_dim]), tf.reshape(tf.slice(u_embed_trans, [i, 0, 0], [1, -1, -1]), [feature_dim, embedding_u_size])) for i in range(edge_type_count)], axis=0)
            node_type_embed = tf.transpose(tf.reduce_mean(tf.reshape(node_embed_tmp, [edge_type_count, -1, neighbor_samples, embedding_u_size]), axis=2), perm=[1,0,2])
        else:
            node_embed_neighbors = tf.nn.embedding_lookup(node_type_embeddings, node_neigh)
            node_embed_tmp = tf.concat([tf.reshape(tf.slice(node_embed_neighbors, [0, i, 0, i, 0], [-1, 1, -1, 1, -1]), [1, -1, neighbor_samples, embedding_u_size]) for i in range(edge_type_count)], axis=0)
            node_type_embed = tf.transpose(tf.reduce_mean(node_embed_tmp, axis=2), perm=[1,0,2])

        trans_w = tf.nn.embedding_lookup(trans_weights, train_types)
        trans_w_s1 = tf.nn.embedding_lookup(trans_weights_s1, train_types)
        trans_w_s2 = tf.nn.embedding_lookup(trans_weights_s2, train_types)

        attention = tf.reshape(tf.nn.softmax(tf.reshape(tf.matmul(tf.tanh(tf.matmul(node_type_embed, trans_w_s1)), trans_w_s2), [-1, u_num])), [-1, att_head, u_num])
        node_type_embed = tf.matmul(attention, node_type_embed)
        node_embed = node_embed + tf.reshape(tf.matmul(node_type_embed, trans_w), [-1, embedding_size])

        if feature_dic is not None:
            node_feat = tf.nn.embedding_lookup(node_features, train_inputs)
            node_embed = node_embed + tf.matmul(node_feat, feature_weights)

        last_node_embed = tf.nn.l2_normalize(node_embed, axis=1)

        loss = tf.reduce_mean(
            tf.nn.nce_loss(
                weights=nce_weights,
                biases=nce_biases,
                labels=train_labels,
                inputs=last_node_embed,
                num_sampled=num_sampled,
                num_classes=num_nodes))
        plot_loss = tf.summary.scalar("loss", loss)

        # Optimizer.
        optimizer = tf.train.AdamOptimizer().minimize(loss, global_step=global_step)

        # Add ops to save and restore all the variables.
        # saver = tf.train.Saver(max_to_keep=20)

        merged = tf.summary.merge_all(key=tf.GraphKeys.SUMMARIES)

        # Initializing the variables
        init = tf.global_variables_initializer()

    # Launch the graph
    print("Optimizing")

    with tf.Session(graph=graph) as sess:
        writer = tf.summary.FileWriter("./runs/" + log_name, sess.graph) # tensorboard --logdir=./runs
        sess.run(init)

        print('Training')
        g_iter = 0
        best_score = 0
        test_score = (0.0, 0.0, 0.0)
        patience = 0

        # Iterate through epochs for training
        for epoch in range(epochs):
            random.shuffle(train_pairs)
            batches = get_batches(train_pairs, neighbors, batch_size)
            # Initialize tqdm progress bar for visual feedback during training
            data_iter = tqdm(batches,
                            desc="epoch %d" % (epoch),
                            total=(len(train_pairs) + (batch_size - 1)) // batch_size,
                            bar_format="{l_bar}{r_bar}")
            avg_loss = 0.0

            for i, data in enumerate(data_iter):
                feed_dict = {train_inputs: data[0], train_labels: data[1], train_types: data[2], node_neigh: data[3]}
                _, loss_value, summary_str = sess.run([optimizer, loss, merged], feed_dict)

                writer.add_summary(summary_str, g_iter)

                g_iter += 1

                avg_loss += loss_value

                if i % 5000 == 0:
                    post_fix = {
                        "epoch": epoch,
                        "iter": i,
                        "avg_loss": avg_loss / (i + 1),
                        "loss": loss_value
                    }
                    data_iter.write(str(post_fix))

            # After each epoch, evaluate the model on validation and testing datasets
            final_model = dict(zip(edge_types, [dict() for _ in range(edge_type_count)]))
            for i in range(edge_type_count):
                for j in range(num_nodes):
                    final_model[edge_types[i]][index2word[j]] = np.array(sess.run(last_node_embed, {train_inputs: [j], train_types: [i], node_neigh: [neighbors[j]]})[0])

            valid_aucs, valid_f1s, valid_prs = [], [], []
            test_aucs, test_f1s, test_prs = [], [], []
            for i in range(edge_type_count):
                if args.eval_type == 'all' or edge_types[i] in args.eval_type.split(','):
                    tmp_auc, tmp_f1, tmp_pr = evaluate(final_model[edge_types[i]], valid_true_data_by_edge[edge_types[i]], valid_false_data_by_edge[edge_types[i]])
                    valid_aucs.append(tmp_auc)
                    valid_f1s.append(tmp_f1)
                    valid_prs.append(tmp_pr)

                    tmp_auc, tmp_f1, tmp_pr = evaluate(final_model[edge_types[i]], testing_true_data_by_edge[edge_types[i]], testing_false_data_by_edge[edge_types[i]])
                    test_aucs.append(tmp_auc)
                    test_f1s.append(tmp_f1)
                    test_prs.append(tmp_pr)
            print('valid auc:', np.mean(valid_aucs))
            print('valid pr:', np.mean(valid_prs))
            print('valid f1:', np.mean(valid_f1s))

            # Calculate average metrics on testing dataset
            average_auc = np.mean(test_aucs)
            average_f1 = np.mean(test_f1s)
            average_pr = np.mean(test_prs)

            # Check if current validation score is better than the best score so far
            cur_score = np.mean(valid_aucs)
            if cur_score > best_score:
                best_score = cur_score
                test_score = (average_auc, average_f1, average_pr)
                patience = 0
            else:
                patience += 1
                if patience > args.patience:
                    print('Early Stopping')
                    break

        # After training, store the final model embeddings in a DataFrame and save to CSV
        final_modelss=[]
        for i in range(edge_type_count):
          for j in range(num_nodes):
            final_modelss.append([edge_types[i],index2word[j],final_model[edge_types[i]][index2word[j]]])
        df = pd.DataFrame((final_modelss))
        df.to_csv(file_name+'/final_modelss'+str(f_num)+'_d_'+str(embedding_size)+'.csv', header=None, index=None) #, sep=' '
    return test_score




This function loads, trains, and evaluates the model by parsing arguments and loads feature data. After training, it computes metrics like ROC-AUC, PR-AUC, and F1 score, offering insights into model performance.

In [ ]:
#GNN training and output embeddings for each created feature

def generateEmbeddings(input,features,epoch,dimensions):
    args = parse_args()
    file_name = input
    print(args)
    if features is not None:
        feature_dic = load_feature_data(features)
        f_num = str(features[-5])
    else:
        feature_dic = None
        f_num = str("dr")
    log_name = file_name.split('/')[-1] + f'_evaltype_{args.eval_type}_b_{args.batch_size}_e_{epoch}'

    training_data_by_type = load_training_data(file_name + '/train.txt')
    valid_true_data_by_edge, valid_false_data_by_edge = load_testing_data(file_name + '/valid.txt')
    testing_true_data_by_edge, testing_false_data_by_edge = load_testing_data(file_name + '/test.txt')

    average_auc, average_f1, average_pr = train_model(training_data_by_type, feature_dic, log_name + '_' + time.strftime('%Y-%m-%d %H-%M-%S',time.localtime(time.time())),f_num,file_name)

    print('Overall ROC-AUC:', average_auc)
    print('Overall PR-AUC', average_pr)
    print('Overall F1:', average_f1)


Below we run our GNN training and output the embedding for each feature. Overall, the process took around 10 hours to complete.

In [ ]:
#Already run
#generateEmbeddings("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5","/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/featuers_m1.txt",1,32)

In [ ]:
#Already Run
#generateEmbeddings("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5","/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/featuers_m2.txt",1,32)

In [ ]:
#Already Run
#generateEmbeddings("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5","/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/featuers_m3.txt",1,32)

In [ ]:
#Already Run
#generateEmbeddings("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5","/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/featuers_m4.txt",1,32)


After the completion of the Graph Neural Network (GNN) runs, the output embeddings generated from these runs are concatenated and utilized as features in a Deep Neural Network (DNN) model for Drug-Drug Interaction (DDI) prediction.


In [ ]:
#Do not run, final files in Evaluation Section
from pandas import DataFrame
import numpy as np
import pandas as pd
import csv

x1=np.array(np.array(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/final_modelss1_d_32.csv", header=None , sep=',')).tolist())
x2=np.array(np.array(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/final_modelss2_d_32.csv", header=None , sep=',')).tolist())
x3=np.array(np.array(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/final_modelss3_d_32.csv", header=None , sep=',')).tolist())
x4=np.array(np.array(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/final_modelss4_d_32.csv", header=None , sep=',')).tolist())


def chang_to_array(s):
  # print(s.shape)
  final_model = []
  # print(final_model.shape)
  for i in s:
    ev = int(i[0])
    dr = int(i[1])
    h = i[2].replace('[',"")
    h = h.replace('...',"")
    h = h.replace('\n',"")
    h = h.replace(']',"")
    h = h.replace('  '," ")
    h = h.replace('  '," ")
    h = h.replace('  '," ")
    h = h.replace('  '," ")
    h = h.replace('  '," ")
    h = h.replace('  '," ")
    h = h.split(" ")
    # h.pop(0)
    for dd,d in enumerate(h):
      try:
        d = float(d)
      except:
        h.remove(d)
    h = np.array(h, dtype=np.float64)
    # print("ff : ",final_model[i,j])
    con = np.concatenate(([ev,dr],h))
    final_model.append(con)
  return np.array(final_model)

x1 = chang_to_array(x1)
print(x1.shape)
x2 = chang_to_array(x2)
print(x2.shape)
x3 = chang_to_array(x3)
print(x3.shape)
x4 = chang_to_array(x4)
print(x4.shape)

print(int(x1[0][1]))
print(len(x1[0,:]))
print(np.array(x1[0,:]))

def reduc_shape(m):
  r = []
  for i in range(572):
    try:
      s2=np.where(m[:,1]==i)[0]
      dd = m[s2[0],2:]
      for j in s2[1:]:
        # dd = np.mean((dd,m[j,2:]), axis=0)
        dd = np.concatenate((dd,m[j,2:]))
      r.append([i,dd])
    except:
      print("c")
  return np.array(r)


def make_dic(x):
  s_dic = dict()
  r = []
  for i in range(572):
    try:
      s2=np.where(x[:,1]==i)[0]
      dd = x[s2[0],2:]
      for j in s2[1:]:
        # dd = np.mean((dd,m[j,2:]), axis=0)
        dd = np.concatenate((dd,x[j,2:]))
      # r.append([i,dd])
      s_dic[i] = dd
    except:
      print("c")
  return s_dic

xs1 = (make_dic(x1))
xs2 = (make_dic(x2))
xs3 = (make_dic(x3))
xs4 = (make_dic(x4))
print(len(xs1[0]))

all_fectuer = []
all_fectuer.append(xs1)
all_fectuer.append(xs2)
all_fectuer.append(xs3)
all_fectuer.append(xs4)
all_fectuer = np.array(all_fectuer)
print(all_fectuer.shape)
print(len(all_fectuer[0]))
print(len(all_fectuer[0][0]))
print(all_fectuer[0][0][0])
# print(all_fectuer[0][336][0])
print(type(all_fectuer[0]))

full_pos=np.array(np.array(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/full_pos2.txt", header=None , sep=' ')).tolist())
print(full_pos.shape)

DDI = full_pos[:,1:3]
print(DDI.shape)
print(65*570)

print(all_fectuer.shape)
f_i1 = all_fectuer[0]
f_i2 = all_fectuer[1]
f_i3 = all_fectuer[2]
f_i4 = all_fectuer[3]

new_feature1 = ( np.array(np.multiply(f_i1[d[0]],f_i1[d[1]])).tolist() for d in (DDI) )
new_feature2 = ( np.array(np.multiply(f_i2[d[0]],f_i2[d[1]])).tolist() for d in (DDI) )
new_feature3 = ( np.array(np.multiply(f_i3[d[0]],f_i3[d[1]])).tolist() for d in (DDI) )
new_feature4 = ( np.array(np.multiply(f_i4[d[0]],f_i4[d[1]])).tolist() for d in (DDI) )

df = pd.DataFrame(np.array(list(new_feature1)))
df.to_csv('/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/t_c_m_1_32.txt', header=None, index=None, sep=' ')
df = pd.DataFrame(np.array(list(new_feature2)))
df.to_csv('/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/t_c_m_2_32.txt', header=None, index=None, sep=' ')
df = pd.DataFrame(np.array(list(new_feature3)))
df.to_csv('/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/t_c_m_3_32.txt', header=None, index=None, sep=' ')
df = pd.DataFrame(np.array(list(new_feature4)))
df.to_csv('/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/t_c_m_4_32.txt', header=None, index=None, sep=' ')

print(len(all_fectuer[0][0]),len(DDI))
full_pos = 0
x1 ,x2 ,x3 ,x4 ,xx1 ,xx2 ,xx3 ,xx4,xs1 ,xs2 ,xs3 ,xs4 = 0,0,0,0,0,0,0,0,0,0,0,0
full_dataframe,f_dataframe,featuers,drugs,a,x=0,0,0,0,0,0

In [ ]:
#Concat() was run to create /content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/t_c_m_{1-4}_32.txt
#concat()

After aggregating the representation vectors, it's time to pass the aggregated vectors into the deep neural network learning model for predictions.

### Deep Neural Network


In [ ]:
# this is the deep neural network learning model

def DNN():
    train_input = Input(shape=(vector_size,), name='Inputlayer')
    train_in = Dense(512, activation='relu')(train_input)
    train_in = BatchNormalization()(train_in)
    train_in = Dropout(droprate)(train_in)
    train_in = Dense(256, activation='relu')(train_in)
    train_in = BatchNormalization()(train_in)
    train_in = Dropout(droprate)(train_in)
    train_in = Dense(event_num)(train_in)
    out = Activation('softmax')(train_in)
    model = Model(inputs=train_input, outputs=out)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) # binary_crossentropy
    return model

# Training

In this paper [1], the DNN model is trained with 5-fold cross validation techique with 4 subsets for training and 1 subset for testing. In the training process, the Adam Optimizer is uesd as optimizer function, and the Categorical cross-entropy loss is used as loss function.

- Hyperparameters -
    - Optimizer: Adam Optimizer
    - Learning Rate: Set to 0.001
    - Batch Size: The batch size used for training is 128.
    - Hidden Layer Sizes: The neural network architecture consists of two hidden layers with 512 and 256 units respectively.
    - Dropout Rate: Dropout rate is set to 0.3 and passed to the DNN() model.
    
- Computational Requirements -
    - Number of Training Epochs: Set to 100.
    - Average runtime for each epoch: the runtime for each epoch varies from 8s to 12s, with an averaged 9s.
    - Total number of trials: 5 trials in training with cross validation.

In [ ]:
def cross_validation(feature_matrix, label_matrix, clf_type, event_num, seed, CV):
    all_eval_type = 11
    result_all = np.zeros((all_eval_type, 1), dtype=float)
    each_eval_type = 6
    result_eve = np.zeros((event_num, each_eval_type), dtype=float)
    y_true = np.array([])
    y_pred = np.array([])
    y_score = np.zeros((0, event_num), dtype=float)
    index_all_class = get_index(label_matrix, event_num, seed, CV)
    matrix = []
    if type(feature_matrix) != list:
        matrix.append(feature_matrix)
        feature_matrix = matrix

    for k in range(CV):
        print("k : ",k)
        train_index = np.where(index_all_class != k)
        test_index = np.where(index_all_class == k)
        pred = np.zeros((len(test_index[0]), event_num), dtype=float)
        # dnn=DNN()
        for i in range(len(feature_matrix)):
            print("f : ",i)
            xx = bring_f(str(feature_matrix[i]))
            xx = np.array(xx)
            x_train = xx[train_index]
            x_test = xx[test_index]
            xx = 0
            y_train = label_matrix[train_index]
            # one-hot encoding
            y_train_one_hot = np.array(y_train)
            y_train_one_hot = (np.arange(y_train_one_hot.max() + 1) == y_train[:, None]).astype(dtype='float32')
            y_test = label_matrix[test_index]
            # one-hot encoding
            y_test_one_hot = np.array(y_test)
            y_test_one_hot = (np.arange(y_test_one_hot.max() + 1) == y_test[:, None]).astype(dtype='float32')
            if clf_type == 'DDIMDL':
                dnn = DNN()
                early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='auto')
                dnn.fit(x_train, y_train_one_hot, batch_size=128, epochs=100,
                        validation_data=(x_test, y_test_one_hot),
                        callbacks=[early_stopping])
                x_train = 0
                pred += dnn.predict(x_test)
                x_test = 0
                continue
            elif clf_type == 'RF':
                clf = RandomForestClassifier(n_estimators=100)
            elif clf_type == 'GBDT':
                clf = GradientBoostingClassifier()
            elif clf_type == 'SVM':
                clf = SVC(probability=True)
            elif clf_type == 'FM':
                clf = GradientBoostingClassifier()
            elif clf_type == 'KNN':
                clf = KNeighborsClassifier(n_neighbors=4)
            else:
                clf = LogisticRegression()
            clf.fit(x_train, y_train)
            pred += clf.predict_proba(x_test)

        dnn = 0
        pred_score = pred / len(feature_matrix)
        pred_type = np.argmax(pred_score, axis=1)
        y_true = np.hstack((y_true, y_test))
        y_pred = np.hstack((y_pred, pred_type))
        y_score = np.row_stack((y_score, pred_score))
    return y_pred, y_score, y_true

In [ ]:
#Full Model Run and Evaluation.
#Do not run, takes ~3 hrs and we already have the output.
from numpy.random import seed
seed(1)
#from tensorflow import set_random_seed
#set_random_seed(2)
import csv
import sqlite3
import time
import numpy as np
import pandas as pd
from pandas import DataFrame
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.metrics import auc
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import label_binarize
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from keras.models import Model
from keras.layers import Dense, Dropout, Input, Activation, BatchNormalization, LSTM, MaxPooling1D, Conv1D
from keras.callbacks import EarlyStopping
# from tf.keras.utils import plot_model
import tensorflow as tf

full_pos=np.array(np.array(pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/full_pos2.txt", header=None , sep=' ')).tolist())
new_label = []
for i in full_pos:
    new_label.append(i[0])
print('new_label : ',len(new_label),(new_label[0]))
DDI = full_pos[:,1:3]
new_label = np.array(new_label)

event_num = 65
droprate = 0.3
vector_size = 2080
clf = "DDIMDL"
CV = 5
seed = 0
f_matrix = [1,2,3,4]
featureName = "/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/G_allf_32_cm"

def get_index(label_matrix, event_num, seed, CV):
    index_all_class = np.zeros(len(label_matrix))
    for j in range(event_num):
        index = np.where(label_matrix == j)
        kf = KFold(n_splits=CV, shuffle=True, random_state=seed)
        k_num = 0
        for train_index, test_index in kf.split(range(len(index[0]))):
            index_all_class[index[0][test_index]] = k_num
            k_num += 1

    return index_all_class

def bring_f(f_item):
  full_dataframe = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/t_c_m_"+f_item+"_32.txt", header=None , sep=' ')
  x1=np.array(np.array(full_dataframe).tolist())
  full_dataframe = 0
  print(x1.shape)
  return x1.tolist()

# Evaluation

The metrics used in this project [1] include ACC, AUPR, AUC, F1 Score, Precision, and Recall. Among these metrics, AUPR and AUC use the micro metrics and the rest use macro metrics.

In [ ]:
def evaluate(pred_type, pred_score, y_test, event_num):
    all_eval_type = 11
    result_all = np.zeros((all_eval_type, 1), dtype=float)
    each_eval_type = 6
    result_eve = np.zeros((event_num, each_eval_type), dtype=float)
    y_one_hot = label_binarize(y_test, classes=np.arange(event_num))
    pred_one_hot = label_binarize(pred_type, classes=np.arange(event_num))

    precision, recall, th = multiclass_precision_recall_curve(y_one_hot, pred_score)

    result_all[0] = accuracy_score(y_test, pred_type)
    result_all[1] = roc_aupr_score(y_one_hot, pred_score, average='micro')
    result_all[2] = roc_aupr_score(y_one_hot, pred_score, average='macro')
    result_all[3] = roc_auc_score(y_one_hot, pred_score, average='micro')
    result_all[4] = roc_auc_score(y_one_hot, pred_score, average='macro')
    result_all[5] = f1_score(y_test, pred_type, average='micro')
    result_all[6] = f1_score(y_test, pred_type, average='macro')
    result_all[7] = precision_score(y_test, pred_type, average='micro')
    result_all[8] = precision_score(y_test, pred_type, average='macro')
    result_all[9] = recall_score(y_test, pred_type, average='micro')
    result_all[10] = recall_score(y_test, pred_type, average='macro')
    for i in range(event_num):
        result_eve[i, 0] = accuracy_score(y_one_hot.take([i], axis=1).ravel(), pred_one_hot.take([i], axis=1).ravel())
        result_eve[i, 1] = roc_aupr_score(y_one_hot.take([i], axis=1).ravel(), pred_one_hot.take([i], axis=1).ravel(),
                                          average=None)
        result_eve[i, 2] = roc_auc_score(y_one_hot.take([i], axis=1).ravel(), pred_one_hot.take([i], axis=1).ravel(),
                                         average=None)
        result_eve[i, 3] = f1_score(y_one_hot.take([i], axis=1).ravel(), pred_one_hot.take([i], axis=1).ravel(),
                                    average='binary')
        result_eve[i, 4] = precision_score(y_one_hot.take([i], axis=1).ravel(), pred_one_hot.take([i], axis=1).ravel(),
                                           average='binary')
        result_eve[i, 5] = recall_score(y_one_hot.take([i], axis=1).ravel(), pred_one_hot.take([i], axis=1).ravel(),
                                        average='binary')
    return [result_all, result_eve]

In [ ]:
#Do Not run - long run time
start = time.perf_counter()

y_pred, y_score, y_true = cross_validation(f_matrix, new_label, clf, event_num, seed, CV)
all_result, each_result = evaluate(y_pred, y_score, y_true, event_num)
print("all_result, each_result \n",all_result,'\n', each_result)
save_result(featureName, 'all', clf, all_result)
save_result(featureName, 'each', clf, each_result)

print("time used:", time.process_time() - start)


# Results - (All code runnable)

Table of Results -

<img src="https://drive.google.com/uc?export=view&id=1nScqXKxCSYpIfh_aIRAWu_jz_qsoJ_c9" width="900" height="350">


**Results for tested Hypothesis:**
  
  **1. Impact of Embedding Dimension Size**
  - Original Hypothesis: We hypothesized that the size of the embedding dimension would affect the model's performance, with a dimension size of 32 leading to the best accuracy.
  - Reproduction Method: To test this, we trained the model using embedding dimension sizes of 16, 32, and 64, mirroring the original paper's approach.
  - Evaluation: After evaluating the model's performance, we found that our results differed with the original hypothesis. They had a dimension size of 32 leading to the highest accuracy, while 16 and 32 were nearly exchangeable for us.
    
  - In this hypothesis, we generated the embedding vectors with different dimensions, including 16, 32, 64. The running for embedding generation under each dimension took 8 to 10 hours. The embedding vectors were stored in the following files:
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/final_modelss{1-4}_d_16.csv"`
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/final_modelss{1-4}_d_32.csv"`
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/final_modelss{1-4}_d_64.csv"`


  - After the embedding generation, the embedding vectors were concatenated, this step took around 1 hour for arregation under each dimension. The resulted aggregated representation vactors were stored in the following files:
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/t\_c\_m\_{1-4}\_16.txt"`
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/t\_c\_m\_{1-4}\_32.txt"`
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/t\_c\_m\_{1-4}\_64.txt.csv"`


  The {1-4} in each filename stands for the for feature matrices used in generating the embedding, including "Target", "Enzyme", "Pathway", "Smile", respectively.

  - The prediction results are stored in the following files:
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/G\_allf\_16\_cm\_all\_DDIMDL.csv"`
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/G\_allf\_32\_cm\_all\_DDIMDL.csv"`
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/G\_allf\_64\_cm\_all\_DDIMDL.csv`


In [ ]:
#Already run
#generateEmbeddings("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5","/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/featuers_m1.txt",1,16)

#generateEmbeddings("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5","/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/featuers_m2.txt",1,16)

#generateEmbeddings("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5","/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/featuers_m3.txt",1,16)

#generateEmbeddings("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5","/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/featuers_m3.txt",1,16)

#Concat() was run to create /content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/t_c_m_{1-4}_16.txt

#generateEmbeddings("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5","/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/featuers_m1.txt",1,64)

#generateEmbeddings("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5","/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/featuers_m2.txt",1,64)

#generateEmbeddings("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5","/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/featuers_m3.txt",1,64)

#generateEmbeddings("/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5","/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/featuers_m3.txt",1,64)

#Concat() was run to create /content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/t_c_m_{1-4}_64.txt

In [ ]:
# Run train_16.py to train the model and make predictions with the embedding size of 16
# Resutls stored in "/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/G_allf_16_cm_all_DDIMDL.csv"

#!python  /content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/train_16.py

In [ ]:
# Run train_64.py to train the model and make predictions with the embedding size of 64, results stored in
# Resutls stored in "/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/G_allf_64_cm_all_DDIMDL.csv"

#!python  /content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/train_64.py

In [ ]:
# Visualize the results
import pandas as pd
import matplotlib.pyplot as plt

root_path = "/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/"
# Load the data from CSV files
file_paths = [
    "G_allf_16_cm_all_DDIMDL.csv",
    "G_allf_32_cm_all_DDIMDL.csv",
    "G_allf_64_cm_all_DDIMDL.csv"
]

dfs = []
for file_path in file_paths:
    df = pd.read_csv(root_path+file_path, header=None)
    dfs.append(df)

# Define column names
columns = ["ACC", "AUPR Micro", "AUPR Macro", "AUC Micro", "AUC Macro", "F1 Micro", "F1 Macro", "Pre Micro", "Pre Macro", "Recall Micro", "Recall Macro"]

# Reshape the data and combine into a single DataFrame
results = pd.DataFrame({f"Dimension {i}": df[0].values for i, df in zip([16, 32, 64], dfs)}, index=columns)

# Plot the results
plt.figure(figsize=(15, 6))
results.plot(kind='bar', rot=45)
plt.title("Comparison of Metrics for Different Embedding Dimensions")
plt.xlabel("Metrics")
plt.ylabel("Value")
plt.legend(title="Embedding Dimension", bbox_to_anchor=(1, 1), loc='upper left')  # Move legend outside

plt.tight_layout()
plt.show()


From the above results of the proposed model run on representation vectors with different embedding size, our results differ slightly with the paper's. They claim 32 dimensions were far better than 16, while both are comparable in our results. The dimension of 64 of embedding size did generate the lowest Accuracy which is compatible with the paper.

**Results for tested Hypothesis:**

**2. Impact of Different Drug Feature Matrices**
- Original Hypothesis: We posited that utilizing different drug feature matrices would lead to varying performance on the proposed model, with the combined feature matrix showing the best performance.
- Reproduction Method: To test this hypothesis, we examined the model's performance using different drug feature matrices, including combined feature matrices.
- Evaluation: Our results confirmed the hypothesis, showing that the combined feature matrix indeed contributed to the highest predictive performance compared to other matrices.

To test this hypothesis, we tried sigle feature matrices of "Target", "Enzyme", "Pathway", "Smile" and combinations of them for test.

  - The prediction results with different feature matrix are stored in the following files:
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/G_allf_32_cm_all_DDIMDL_T.csv"`
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/G_allf_32_cm_all_DDIMDL_E.csv"`
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/G_allf_32_cm_all_DDIMDL_P.csv`
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/G_allf_32_cm_all_DDIMDL_S.csv`



In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

root_path = "/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/"

# Load data
files = ['G_allf_32_cm_all_DDIMDL_S.csv', 'G_allf_32_cm_all_DDIMDL_E.csv',
         'G_allf_32_cm_all_DDIMDL_T.csv', 'G_allf_32_cm_all_DDIMDL_P.csv',
         'G_allf_32_cm_all_DDIMDL_E_P.csv', 'G_allf_32_cm_all_DDIMDL_E_S.csv',
         'G_allf_32_cm_all_DDIMDL_E_T.csv', 'G_allf_32_cm_all_DDIMDL_P_S.csv',
         'G_allf_32_cm_all_DDIMDL_E_P_S.csv', 'G_allf_32_cm_all_DDIMDL_E_P_T.csv',
         'G_allf_32_cm_all_DDIMDL_E_S_T.csv', 'G_allf_32_cm_all_DDIMDL_P_S_T.csv',
         'G_allf_32_cm_all_DDIMDL.csv']

metrics = ['ACC', 'AUPR Micro', 'F1 Macro']
colors = ['black', 'blue', 'red']  # Define colors for each metric

# Order of metrics in the files
metric_order = ['ACC', 'AUPR Micro', 'AUPR', 'AUC Micro', 'AUC Macro', 'F1 Micro', 'F1 Macro', 'Precision Micro', 'Precision Macro', 'Recall Micro', 'Recall Macro']

plt.figure()  # Create a single plot for all metrics
plt.title("Metrics Comparison")  # Set title for the plot
plt.xlabel('Feature Matrix')  # Set x-axis label
plt.ylabel('Value')  # Set y-axis label

legend_handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='black', markersize=8),
                  plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='blue', markersize=8),
                  plt.Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=8)]

for metric, color in zip(metrics, colors):
    metric_values = []
    for file in files:
        # Load data from CSV
        df = pd.read_csv(root_path + file, header=None)  # Assuming no header in files
        metric_index = metric_order.index(metric)  # Find index of the metric in order
        metric_values.append(df.iloc[metric_index].values)

    # Plot the data for all files for this metric
    for file, values in zip(files, metric_values):
        plt.plot(['GD+S', 'GD+E', 'GD+T', 'GD+P', 'GD+E+P', 'GD+E+S', 'GD+E+T', 'GD+P+S', 'GD+E+P+S','GD+E+P+T','GD+E+S+T','GD+P+S+T', 'GD+S+E+T+P'], metric_values, label=file, color=color, marker='o')

plt.legend(legend_handles, ['ACC', 'AUPR', 'F1'], loc='lower right')
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.show()  # Show the plot with all lines


In the above results of the proposed model run on different feature matrix, including individual matrix 'S', 'E', 'T', 'P', and different combination of feature matrices. We can tell as adding more and more feature matrics, the model'e performance is inceased. When four of the feature matrics are applied in the proposed model, the performance is the best. This is compatible with the paper.

**Results for tested Hypothesis:**

**3. Comparison with Existing Models:**
- Original Hypothesis: We expected that our proposed method would outperform existing models such as DNN, RF, KNN, and LR in predicting DDIs.
- Reproduction Method: To validate this hypothesis, we compared the performance of our proposed method with these baseline models using common performance metrics.
- Evaluation: Our comparative analysis demonstrated that the proposed method indeed surpassed existing models in predicting DDIs, consistent with our original hypothesis.

In our reproducing, we didn't try models like MDNN, CNN-DDI, DANN_DDI, DDIMDL, DeepDDI, DNN which were introduced in other papers. Instead, we chose RF, KNN, LR which can be directly called from sklearn library for comparison.

  - The prediction results with different models are stored in the following files:
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/G_allf_32_cm_all_DDIMDL.csv"`
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/G_allf_32_cm_all_RF.csv"`
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/G_allf_32_cm_all_KNN.csv`
- `"/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/G_allf_32_cm_all_LR.csv`

In [ ]:
# Visualize the results
import pandas as pd
import matplotlib.pyplot as plt

root_path = "/content/drive/MyDrive/Colab Notebooks/GNN-DDI/GNN_DDI/DDI/data5/"
# Load the data from CSV files
file_paths = [
    "G_allf_32_cm_all_DDIMDL.csv",
    "G_allf_32_cm_all_RF.csv",
    "G_allf_32_cm_all_KNN.csv",
    "G_allf_32_cm_all_LR.csv"
]

dfs = []
for file_path in file_paths:
    df = pd.read_csv(root_path+file_path, header=None)
    dfs.append(df)

# Define column names
columns = ["ACC", "AUPR Micro", "AUPR Macro", "AUC Micro", "AUC Macro", "F1 Micro", "F1 Macro", "Pre Micro", "Pre Macro", "Recall Micro", "Recall Macro"]

# Reshape the data and combine into a single DataFrame
results = pd.DataFrame({f"{i}": df[0].values for i, df in zip(['GNN_DDI', 'RF', 'KNN', 'LR'], dfs)}, index=columns)

# Plot the results
plt.figure(figsize=(12, 10))
results.plot(kind='bar', rot=45)
plt.title("Comparison of Metrics for Different Models")
plt.xlabel("Metrics")
plt.ylabel("Value")
plt.legend(title="Model", bbox_to_anchor=(1, 1), loc='upper left')  # Move legend outside
plt.tight_layout()
plt.show()


From the above results of different models, including the proposed model, running on the embedding vectors with dimension 32, we can tell the proposed model provides the best result, with the highest value on every metric. It's notable that the Logistic Regression has difficulty converging during the training and generated the worst performance.

# Discussion

Reproducing the paper's results predominantly relied on the GitHub code provided by the authors. Leveraging their codebase significantly expedited the replication process, as it served as a comprehensive blueprint for implementing the proposed methodology.

The GitHub repository contained the necessary scripts, configuration files, and data, which greatly facilitated the setup and execution of the experiments. Having access to the original implementation allowed for a more straightforward interpretation of the methodology described in the paper.

What made the reproduction process relatively easy was the clarity and organization of the GitHub repository. The code was well-documented, with comments explaining the purpose of each function and module. This made it easier to navigate through the codebase and understand its structure.

Additionally, the availability of pre-trained models and example datasets enabled quick validation of the code and ensured consistency with the paper's reported results. This provided a solid foundation for building upon and extending the experiments as needed.

However, despite the advantages of using the GitHub code, there were still challenges encountered during the reproduction process. Adapting the code to suit google colab and data required careful consideration and sometimes necessitated modifications to the original implementation. Furthermore, troubleshooting issues related to software dependencies, compatibility with different environments, and hardware constraints posed additional hurdles along the way.

In summary, while the GitHub code provided a valuable starting point for reproducing the paper's results, the process was not entirely devoid of challenges. Nevertheless, leveraging the provided codebase significantly expedited the replication process and contributed to the overall reproducibility of the experiments.

To improve reproducibility, we would suggest the following to the authors or other reproducers:

1. Ensure that all necessary code files and functions are provided with clear documentation on how to use them.
2. Provide detailed instructions on how to integrate any additional models or components into the existing codebase. Clear guidance can significantly improve the reproducibility of results.


# References

1.   Al-Rabeah, M.H., Lakizadeh, A. Prediction of drug-drug interaction events using graph neural networks based feature extraction. Sci Rep 12, 15590 (2022). https://doi.org/10.1038/s41598-022-19999-4